In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from statsmodels.tsa.arima_model import ARIMA
import seaborn as sns
import matplotlib.pyplot as plt
import arima
import datetime
import time
%matplotlib inline
from sklearn.decomposition import PCA
import stl

import sys
sys.path.append('../')
from pipeline import *
from onehot import *
from util import *

# load data
train_data = pd.read_csv('raw_data/train.csv')
test_data = pd.read_csv('raw_data/test.csv')

d:\app\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def select_arima_model(sales, orders, factors=None,speedup=False):
    best_r = None
    best_o = None
#     min_aic=9999999999999
    min_bic=9999999999999
        
    for o in orders:
        try:
            if speedup:
                m=sm.tsa.statespace.SARIMAX(sales,order=o,exog=factors,
                         simple_differencing=True, enforce_stationarity=False, enforce_invertibility=False)
                    
            else:
                m=sm.tsa.statespace.SARIMAX(sales,order=o,exog=factors,enforce_stationarity=False)
                
            r=m.fit(disp=False)
            if r.bic < min_bic:
                best_r = r
                best_o = o
#                 min_aic=r.aic
                min_bic=r.bic
#             print(o,r.bic)

        except Exception as e:
            pass
#             print('%s %s'% (o,e))
#             traceback.print_exc()
            
    return best_r,best_o

def make_orders(range_num, seq_num):
    if seq_num == 0:
        return [[]]
    else:
        orders=[]
        sub_orders=make_orders(range_num,seq_num-1)
        for o in sub_orders:
            for i in range(range_num):
                s=o.copy()
                s.append(i)
                orders.append(s)
        return orders

t=time.time()
preds=[]
depts=sorted(test_data.Dept.unique())
# iterate every dept
for dept in depts:
#     if dept < 6:
#         continue
    print('dept %d' % dept)
    # transform train to dept, stores(in columns) matrix, index is date fill na with 0
    dept_data = train_data[train_data.Dept == dept][['Date','Store','Weekly_Sales']]
    if len(dept_data) == 0:
        print('no dept data')
        continue
#     dept_stores = pd.pivot_table(dept_data, values='Weekly_Sales', index='Date', columns='Store', fill_value=0.)
#     dept_stores = pd.DataFrame(dept_stores.values,index=pd.DatetimeIndex(dept_stores.index),columns=dept_stores.columns)
    
    # pca preprocess the sales between stores to filter noise ,use 12 components    
#     idxs=dept_stores.index
#     cols=dept_stores.columns
#     pca = PCA(n_components=12)
#     dept_pca = pca.fit_transform(dept_stores)
#     dept_restore = pca.inverse_transform(dept_pca)
#     dept_stores = pd.DataFrame(dept_restore,index=pd.DatetimeIndex(idxs),columns=cols)
    
    # iterate every store
    stores = sorted(test_data[test_data.Dept == dept].Store.unique())
    for store in stores:
#         if store != 42:
#             continue
        
        store_test=test_data[np.logical_and(test_data.Store==store,test_data.Dept==dept)][['Date']]
        
        if len(store_test)>0:
            print('store %d'%store)
            store_test['Id']=str(store)+'_'+str(dept)+'_'+store_test.Date.astype('U')
            store_test['Weekly_Sales']=0
            store_test=pd.DataFrame(store_test[['Id','Weekly_Sales']].values,
                                    index=pd.DatetimeIndex(store_test.Date),columns=['Id','Weekly_Sales'])
            
#             store_train = dept_stores.loc[:,store]
            store_train = pd.DataFrame(dept_data[dept_data.Store==store].Weekly_Sales.values,
                                      index=pd.DatetimeIndex(dept_data[dept_data.Store==store].Date),
                                      columns=['Weekly_Sales'])
            
            
            if len(store_train) > 1:
                # fill train missing date sales
                store_resample = store_train.resample('1W').sum()
                store_resample = store_resample.fillna(0)
                store_resample.index -= store_resample.index[0]-store_train.index[0]
                store_train=store_resample
                
                # stl the ts
                try:
                    de = stl.seasonal_decompose(store_train, freq=52)

                    # select arima for trend comp
                    orders = make_orders(2,3)
                    m,o=select_arima_model(de.trend.values,orders)
                    if m is not None:
                        print('best %s %s',o,m.bic)

                        # forecast until test end date

                        # test dates may not successive with train dates , and may not continues
                        pred_dates=np.array([None]*2)
                        pred_dates[0]=store_train.index[-1]+np.timedelta64(1,'W')
                        pred_dates[1]=store_test.index[-1]
                        pred_data=pd.DataFrame(index=pd.Series(pred_dates))
                        pred_data=pred_data.resample('1W').sum()
                        pred_data['Weekly_Sales']=0
                        # resample change dates adjustment, need change back
                        pred_data.index = pred_data.index - (pred_data.index[0]-pred_dates[0])

                        # predict continous dates
                        fc_data=m.forecast(len(pred_data))
                        fc_data=fc_data.reshape(fc_data.shape[0],1)


                        # add last season comp to forecasts
                        if len(de.seasonal) >= 52 and len(fc_data)<=52:
                            fc_data+=de.seasonal.iloc[-52:-52+len(fc_data)].values
                        pred_data.Weekly_Sales=fc_data

                        # write back to test data
                        for date in store_test.index:
                            sale_data = pred_data[pred_data.index == date]
                            store_test.loc[date,'Weekly_Sales'] = int(sale_data.loc[date,'Weekly_Sales'])
                except Exception as e:
                    print(e)

            preds.append(store_test[['Id','Weekly_Sales']])
#         break
#     break

print('total time %ds'%int(time.time()-t))
        
# submit
result=pd.concat(preds)
result.to_csv('output/stl/result.csv',index=False,header=True)


dept 1
store 1


d:\app\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


best %s %s [1, 1, 1] 1099.66718143
store 2
best %s %s [1, 1, 1] 1181.75989453
store 3


d:\app\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


best %s %s [1, 1, 1] 875.112898447
store 4
best %s %s [1, 1, 1] 1192.49620598
store 5
best %s %s [1, 1, 1] 885.468119216
store 6
best %s %s [1, 1, 0] 1179.26055063
store 7
best %s %s [1, 1, 0] 931.956918235
store 8

...

best %s %s [1, 1, 1] 600.491953941
store 28
best %s %s [1, 1, 1] 513.944427488
store 29
store 30


d:\app\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
d:\app\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


best %s %s [0, 1, 1] -554.934378018
store 31
best %s %s [1, 1, 1] 567.689262417
store 32
best %s %s [1, 1, 1] 560.259349732
store 33
operands could not be broadcast together with shapes (10,) (96,) 
store 34
best %s %s [1, 1, 1] 516.768504424
store 36
store 37
operands could not be broadcast together with shapes (36,) (70,) 
store 38
store 39
best %s %s [1, 1, 1] 515.020569207
store 40
best %s %s [0, 1, 1] 110.534937779
cannot convert float NaN to integer
store 41


d:\app\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


best %s %s [0, 1, 1] 176.142260956
cannot convert float NaN to integer
total time 1022s


lowess 3789

ma 3233